In [8]:
import torch
import numpy as np
import cv2
from torchvision import transforms
from einops import rearrange
import imageio

# llava_pythia / robosuite
from llava_pythia.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN
from llava_pythia.mm_utils import tokenizer_image_token, get_model_name_from_path
from llava_pythia.model.builder import load_pretrained_model
from llava_pythia.conversation import conv_templates
from llava_pythia.model.language_model.pythia.llava_pythia import LlavaPythiaConfig

import robosuite as suite
from robosuite.controllers import load_composite_controller_config
from robosuite.utils.transform_utils import (
    mat2quat, quat_multiply, quat_inverse, quat2axisangle
)

# ===================== 이미지 준비 =====================
def get_image(ts, camera_names, rand_crop_resize=False):
    # 학습 분포와 동일하게 180x320
    imgs = []
    for cam in camera_names:
        im = ts.observation['images'][cam]
        if (im.shape[0], im.shape[1]) != (180, 320):
            im = cv2.resize(im, (320, 180))
        imgs.append(rearrange(im, 'h w c -> c h w'))
    img_tensor = torch.from_numpy(np.stack(imgs) / 255.0).float().cuda().unsqueeze(0)

    if rand_crop_resize:
        h, w = img_tensor.shape[-2:]
        ratio = 0.95
        dh, dw = int(h * (1 - ratio) / 2), int(w * (1 - ratio) / 2)
        img_tensor = img_tensor[..., dh:h - dh, dw:w - dw].squeeze(0)
        img_tensor = transforms.Resize((h, w), antialias=True)(img_tensor).unsqueeze(0)
    return img_tensor

# ===================== 회전 유틸 (6D -> R, R -> quat wxyz) =====================
def rot6d_to_matrix(rot6d: torch.Tensor) -> torch.Tensor:
    """
    rot6d: (..., 6) -> rotation matrix (..., 3, 3)
    """
    a1 = torch.nn.functional.normalize(rot6d[..., 0:3], dim=-1)
    a2 = rot6d[..., 3:6]
    b2 = torch.nn.functional.normalize(a2 - (a1 * a2).sum(-1, keepdim=True) * a1, dim=-1)
    b3 = torch.cross(a1, b2, dim=-1)
    return torch.stack([a1, b2, b3], dim=-2)  # (...,3,3)

def matrix_to_quat_xyzw(R: torch.Tensor) -> np.ndarray:
    """
    torch (...,3,3) -> numpy (...,4) in (x,y,z,w)
    mat2quat는 (w,x,y,z)를 반환하므로 xyzw로 재정렬
    """
    R_np = R.detach().cpu().numpy()
    q_wxyz = np.asarray([mat2quat(r) for r in R_np])      # (N,4) [w,x,y,z]
    q_xyzw = np.concatenate([q_wxyz[:, 1:4], q_wxyz[:, 0:1]], axis=-1)
    return q_xyzw



# ===================== 정책 래퍼 =====================
class llava_pythia_act_policy:
    def __init__(self, policy_config, data_args=None):
        self.policy_config = policy_config
        self.data_args = data_args
        self._load_policy()

    def _load_policy(self):
        base = self.policy_config["model_base"] if self.policy_config['enable_lora'] else None
        name = get_model_name_from_path(self.policy_config['model_path'])
        path = self.policy_config["model_path"]
        self.tokenizer, self.policy, self.image_processor, self.context_len = load_pretrained_model(
            path, base, name, False, False
        )
        self.config = LlavaPythiaConfig.from_pretrained('/'.join(path.split('/')[:-1]), trust_remote_code=True)

    def _expand2square(self, imgs, bg_color):
        b, c, h, w = imgs.shape
        size = max(h, w)
        canvas = np.full((b, size, size, c), bg_color, dtype=np.float32)
        imgs_np = imgs.permute(0, 2, 3, 1).cpu().numpy()
        if h >= w:
            offset = (size - w) // 2
            canvas[:, :h, offset:offset + w, :] = imgs_np
        else:
            offset = (size - h) // 2
            canvas[:, offset:offset + h, :w, :] = imgs_np
        return torch.tensor(canvas).to(dtype=imgs.dtype, device=imgs.device)

    def process_batch_to_llava(self, curr_image, robo_state, raw_lang):
        self.conv = conv_templates[self.policy_config['conv_mode']].copy()
        curr_image = curr_image.squeeze(0) if curr_image.dim() == 5 else curr_image
        img1, img2, img3 = torch.chunk(curr_image, 3, dim=0)
        states = robo_state.unsqueeze(0) if robo_state.dim() == 1 else robo_state

        def prep(img):
            img = self._expand2square(img, tuple(self.image_processor.image_mean))
            return self.image_processor.preprocess(
                img, return_tensors='pt', do_normalize=True, do_rescale=False, do_center_crop=False
            )['pixel_values'].float().to(self.policy.device)

        image_tensor     = prep(img1)
        image_tensor_r   = prep(img2)
        image_tensor_top = prep(img3)

        prompt = DEFAULT_IMAGE_TOKEN + '\n' + raw_lang
        if self.policy.config.mm_use_im_start_end:
            prompt = DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_TOKEN + DEFAULT_IM_END_TOKEN + '\n' + raw_lang
        self.conv.append_message(self.conv.roles[0], prompt)
        self.conv.append_message(self.conv.roles[1], None)
        prompt = self.conv.get_prompt() + " <|endoftext|>"

        input_ids = tokenizer_image_token(
            prompt, self.tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt'
        ).unsqueeze(0).cuda().long()
        attn_mask = input_ids.ne(self.tokenizer.pad_token_id).long()

        return dict(
            input_ids=input_ids, attention_mask=attn_mask,
            images=image_tensor, images_r=image_tensor_r, images_top=image_tensor_top,
            states=states
        )

# ===================== 배치 환경 =====================
class RobosuiteDeployEnv:
    def __init__(self, env_name="Lift", cameras=("sideview", "sideview_opposite", "robot0_eye_in_hand"), control_freq=20):
        controller_config = load_composite_controller_config(robot="Panda")
        self.env = suite.make(
            env_name=env_name, robots="Panda", controller_configs=controller_config,
            has_renderer=False, has_offscreen_renderer=True, render_camera=None,
            use_object_obs=True, use_camera_obs=True, control_freq=control_freq,
            camera_names=list(cameras), camera_heights=240, camera_widths=320
        )
        self.view_map = {cam: cam for cam in cameras}
        self.logical_camera_names = cameras
        self.reset()

    def reset(self):
        self.obs = self.env.reset()
        return self.obs

    def get_observation(self):
        ts = type("Timestep", (), {})()
        ts.observation = {'images': {cam: self.obs[f'{cam}_image'] for cam in self.env.camera_names}}

        # 관절 7D를 state로 (학습과 동일)
        if 'robot0_joint_pos' in self.obs:
            qpos7 = self.obs['robot0_joint_pos'][:7].copy()
        else:
            qpos7 = self.env.robots[0]._joint_positions[:7].copy()

        return ts, qpos7

    def step(self, action):
        self.obs, reward, done, info = self.env.step(action)
        return self.obs, reward, done, info

    def render_cameras(self, cameras, width=320, height=240):
        self.env.sim.forward()
        mujoco_cam_names = [self.view_map[cam_name] for cam_name in cameras]
        images = []
        for c in mujoco_cam_names:
            img = self.env.sim.render(camera_name=c, width=width, height=height, depth=False)
            images.append(np.flipud(img))
        return images

# ===================== 평가 루프 =====================
def eval_bc(
    policy, deploy_env, policy_config, save_episode=True, num_rollouts=1,
    raw_lang=None, n_steps=50, fps=20,
    camera_names=("sideview", "sideview_opposite", "robot0_eye_in_hand")
):
    assert raw_lang is not None
    all_frames = []

    # ckpt 설정
    ckpt_action_dim = getattr(policy.policy.config, "action_dim", 7)
    state_dim       = getattr(policy.policy.config, "state_dim", 7)
    use_state_ckpt  = bool(getattr(policy.policy.config, "use_state", False))
    head_type       = getattr(policy.policy.config, "action_head_type", "")
    print(f"[ckpt] action_dim={ckpt_action_dim}, state_dim={state_dim}, use_state={use_state_ckpt}, head={head_type}")

    # diffusion이면 역정규화 준비
    import os, pickle
    stats_path = os.path.join(os.path.dirname(policy_config["model_path"]), "dataset_stats.pkl")
    amin = amax = None
    if head_type == "droid_diffusion":
        with open(stats_path, "rb") as f:
            stats = pickle.load(f)
        amin = torch.tensor(stats["action_min"], device="cuda", dtype=torch.float32)
        amax = torch.tensor(stats["action_max"], device="cuda", dtype=torch.float32)

        def denorm_action(a_norm: torch.Tensor) -> torch.Tensor:
            return (a_norm + 1.0) * 0.5 * (amax - amin) + amin

    # 제어 게인 / 클램프
    POS_GAIN = 0.2
    ROT_GAIN = 0.2
    POS_CLIP = 0.03
    ROT_CLIP = 0.25

    # === 그리퍼 파라미터 (로컬 변수로 유지) ===
    GRIP_VEL_CLIP = 0.5
    G_GAIN = 2.0
    GRIP_EMA_ALPHA = 0.2
    grip_ema = None       # ← 전역(global) 쓰지 말고 함수 내부에서 유지

    for rollout_idx in range(num_rollouts):
        deploy_env.reset()

        for t in range(n_steps):
            ts, robot_state = deploy_env.get_observation()

            # 카메라 매핑
            mapped_obs_images = {
                logical_name: ts.observation['images'][mujoco_name]
                for logical_name, mujoco_name in deploy_env.view_map.items()
                if mujoco_name in ts.observation['images']
            }
            ts.observation['images'] = mapped_obs_images

            # state 주입
            if use_state_ckpt:
                robot_tensor = torch.from_numpy(np.asarray(robot_state[:state_dim], dtype=np.float32)).cuda()
            else:
                robot_tensor = torch.zeros(state_dim, dtype=torch.float32, device="cuda")

            # 이미지 텐서
            image_tensor = get_image(ts, camera_names)

            # 배치
            batch = policy.process_batch_to_llava(image_tensor, robot_tensor, raw_lang)

            with torch.no_grad():
                out = policy.policy(**batch, eval=True)             # (1,T,D) or (T,D)
                a_norm = out[0][0] if out.dim() == 3 else out[0]    # 첫 토큰
                a = denorm_action(a_norm) if head_type == "droid_diffusion" else a_norm
                a_np = a.detach().cpu().numpy()

                # === 10D -> 절대 목표 (pos3 + rot6d + grip1) ===
                if ckpt_action_dim == 10:
                    pos_t   = a_np[:3].astype(np.float32)
                    rot6d_t = torch.from_numpy(a_np[3:9]).to("cuda", dtype=torch.float32).unsqueeze(0)
                    R_t     = rot6d_to_matrix(rot6d_t)         # (1,3,3)
                    q_t_xyzw = matrix_to_quat_xyzw(R_t)[0]     # (x,y,z,w)

                    grip_t  = float(a_np[9])

                    # 현재 EEF (robosuite는 관측을 보통 xyzw로 준다)
                    curr_pos  = np.asarray(deploy_env.obs['robot0_eef_pos'],  dtype=np.float32)   # (3,)
                    curr_quat_xyzw = np.asarray(deploy_env.obs['robot0_eef_quat'], dtype=np.float32)  # (x,y,z,w)

                    # === 위치/자세 델타 ===
                    delta_pos = (pos_t - curr_pos) * POS_GAIN

                    # q_delta = q_target ⊗ inv(q_curr)   (모두 xyzw)
                    q_delta_xyzw = quat_multiply(q_t_xyzw, quat_inverse(curr_quat_xyzw))
                    axis_delta = quat2axisangle(q_delta_xyzw) * ROT_GAIN  # (3,)

                    # === 그리퍼: 절대 -> 속도/델타 변환 (EMA로 안정화) ===
                    if 'robot0_gripper_qpos' in deploy_env.obs:
                        gq = np.asarray(deploy_env.obs['robot0_gripper_qpos'], dtype=np.float32)
                        curr_open_frac = float(np.clip(np.mean(gq) / 0.04, 0.0, 1.0))
                    else:
                        curr_open_frac = 0.0

                    # EMA 업데이트 (로컬 변수)
                    if grip_ema is None:
                        grip_ema = grip_t
                    else:
                        grip_ema = (1.0 - GRIP_EMA_ALPHA) * grip_ema + GRIP_EMA_ALPHA * grip_t

                    # 히스테리시스 스냅
                    if abs(grip_ema - 1.0) < 0.05:
                        grip_ema = 1.0
                    elif abs(grip_ema - 0.0) < 0.05:
                        grip_ema = 0.0

                    grip_vel = np.clip((grip_ema - curr_open_frac) * G_GAIN, -GRIP_VEL_CLIP, GRIP_VEL_CLIP)

                    # === 최종 액션 ===
                    action = np.zeros(7, dtype=np.float32)
                    action[:3] = np.clip(delta_pos, -POS_CLIP, POS_CLIP)
                    action[3:6] = np.clip(axis_delta, -ROT_CLIP, ROT_CLIP)
                    action[6]   = float(grip_vel)
                else:
                    # 7D 직접 출력일 때(보통 델타로 가정) — 필요 시 게인/클립
                    action = a_np.astype(np.float32)
                    action[:3] *= POS_GAIN
                    action[3:6] *= ROT_GAIN
                    action[:3] = np.clip(action[:3], -POS_CLIP, POS_CLIP)
                    action[3:6] = np.clip(action[3:6], -ROT_CLIP, ROT_CLIP)
                    action[6]   = np.clip(action[6], -1.0, 1.0)

                if t < 6:
                    print(f"[Step {t:03d}]")
                    print("  a_norm:", np.round(a_norm.detach().cpu().numpy(), 3))
                    print("  a_denorm:", np.round(a_np, 3))
                    if ckpt_action_dim == 10:
                        print("  target pos:", np.round(pos_t, 3))
                        print("  curr pos:", np.round(curr_pos, 3))
                        print("  delta_pos(gain):", np.round(action[:3], 3))
                        print("  axis_delta(gain):", np.round(action[3:6], 3))
                        print("  grip_vel:", round(action[6], 3))


                _, reward, done, _ = deploy_env.step(action)
                print(f"Step {t:03d} | Action: {np.round(action, 3)}")

                frames = deploy_env.render_cameras(cameras=camera_names, width=640, height=480)
                if len(frames) == 3:
                    try:
                        frame_concat = np.concatenate(frames, axis=1)
                        all_frames.append(frame_concat)
                    except Exception as e:
                        print(f"🚨 프레임 연결 실패: {e}")

                if done:
                    break

    if save_episode and all_frames:
        video_path = "rollout_3_views_5.mp4"
        imageio.mimsave(video_path, all_frames, fps=fps)
        print(f"🎥 {video_path} 저장 완료")


In [9]:
# ===================== 실행부 =====================
if __name__ == '__main__':
    policy_config = {
        "model_path": "/home/parkjeongsu/TinyVLA/OUTPUT_llava_pythia_4/checkpoint-20000",
        "model_base": None,
        "enable_lora": False,
        "conv_mode": "pythia",
        "action_head": "droid_diffusion",
        "action_dim": 10,   # ← ckpt에 맞게! (로그에서 10 확인됨)
        "chunk_size": 10
    }

    LOGICAL_CAMERA_NAMES = ("sideview", "sideview_opposite", "robot0_eye_in_hand")
    env = RobosuiteDeployEnv(env_name="Lift", cameras=LOGICAL_CAMERA_NAMES)
    policy = llava_pythia_act_policy(policy_config)

    eval_bc(
        policy, env, policy_config,
        raw_lang="pick the cube up",
        n_steps=100, fps=10,
        camera_names=LOGICAL_CAMERA_NAMES
    )

[robosuite INFO] Loading controller configuration from: /home/parkjeongsu/.local/lib/python3.10/site-packages/robosuite/controllers/config/robots/default_panda.json (composite_controller_factory.py:121)
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


load llaVA-Pythia MLLM!!!
combine layer: Linear(in_features=519, out_features=512, bias=True)
number of parameters: 7.283150e+07


Some weights of the model checkpoint at /home/parkjeongsu/TinyVLA/OUTPUT_llava_pythia_4/checkpoint-20000 were not used when initializing LlavaPythiaForCausalLM: ['lm_head.weight']
- This IS expected if you are initializing LlavaPythiaForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LlavaPythiaForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


{'device_map': 'cuda', 'torch_dtype': torch.float32}
[ckpt] action_dim=10, state_dim=7, use_state=True, head=droid_diffusion
[Step 000]
  a_norm: [ 0.228  0.77   0.257 -0.635 -0.33  -0.208  0.885  0.143  0.368 -0.426]
  a_denorm: [ 0.493  0.246  0.359 -0.198 -0.324 -0.314  0.885 -0.108  0.161  0.287]
  target pos: [0.493 0.246 0.359]
  curr pos: [-0.106  0.006  1.01 ]
  delta_pos(gain): [ 0.03  0.03 -0.03]
  axis_delta(gain): [ 0.159 -0.25   0.25 ]
  grip_vel: 0.5
Step 000 | Action: [ 0.03   0.03  -0.03   0.159 -0.25   0.25   0.5  ]
[Step 001]
  a_norm: [-0.501  0.815 -0.829 -0.441 -0.264 -0.874 -0.128  0.537  0.811  0.013]
  a_denorm: [ 0.2    0.264 -0.07  -0.056 -0.259 -0.832 -0.128  0.2    0.369  0.506]
  target pos: [ 0.2    0.264 -0.07 ]
  curr pos: [-0.103  0.013  1.012]
  delta_pos(gain): [ 0.03  0.03 -0.03]
  axis_delta(gain): [-0.25 -0.25 -0.25]
  grip_vel: 0.5
Step 001 | Action: [ 0.03  0.03 -0.03 -0.25 -0.25 -0.25  0.5 ]
[Step 002]
  a_norm: [-0.894 -0.05  -0.388 -0.52  -0.7

/home/parkjeongsu/.local/lib/python3.10/site-packages/imageio_ffmpeg/_utils.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


🎥 rollout_3_views_5.mp4 저장 완료
